In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

# San Francisco Employee Incomes

In [ ]:
data = 'san_francisco_2015.csv' # 'http://inferentialthinking.com/notebooks/san_francisco_2015.csv'

sf = Table.read_table(data).select(3, 11, 21)
sf.set_format(2, NumberFormatter(0))
sf = sf.where(2, are.above(10000))
sf.show(5)

In [ ]:
sf.sort(2, descending  =  True)

In [ ]:
comp_bins = np.arange(0, 700000, 25000)
sf.hist(2, bins=comp_bins, unit='dollar')

In [ ]:
#suppose we only have access to the sample:

sample_from_population = sf.sample(200, with_replacement=False)  #unique workers 
sample_from_population.show(3)

In [ ]:
comp_bins = np.arange(0, 700000, 25000)
sample_from_population.hist(2, bins=comp_bins, unit='dollar')

What is the median compensation in the population? This is a parameter.
Presumably, you would not know it.

In [ ]:
np.median(sf.column(2))

Use a statistic instead.
Calculate the median of the sample, and use this sample median as an estimate for the population median.

In [ ]:
np.median(sample_from_population.column(2))

back to slides for Variability of the Estimate

## Bootstrap

In [ ]:
sample_from_population.show(3)

In [ ]:
np.median(sample_from_population.column(2))


In [ ]:
# it does the bootstrap for you

resample = sample_from_population.sample()
resample.show(3)


In [ ]:
resample.select("Total Compensation").hist(bins = np.arange(0, 700000, 25000))

In [ ]:
np.median(resample.column(2))

Now we have another estimate for the population median. Do it again, a thousand times!

In [ ]:
resampled_medians = []

for i in np.arange(1000):
    resample = sample_from_population.sample()
    median = np.median(resample.column(2))
    resampled_medians.append(median)

Table().with_column(
    "Resampled median", resampled_medians
).hist(unit='dollar')

What is the difference between the two histograms above?

In [ ]:
# Where is the true population median? We have the luxury of being able to access the full population.
# Normally, only have the sample (and resamples) from which to make a guess.

np.median(sf.column(2))

In [ ]:
Table().with_column('Resampled median', resampled_medians).hist(unit='dollar')
pop_median = np.median(sf.column(2))
plots.scatter(pop_median, 0, color='red', s=400)

In [ ]:
#In yellow: middle 95% interval
#Red Circle: True median

Table().with_column('Resampled median', resampled_medians).hist(0)

# Draw the line and dot
# Ignore the plotting commands.
interval_95 = [percentile(2.5, resampled_medians),
               percentile(97.5, resampled_medians)]
plots.plot(interval_95, [0, 0], color='gold', lw=5)
pop_median = np.median(sf.column(2))
plots.scatter(pop_median, 0, color='red', s=400)

In [ ]:
# pull out the middle 95%
# need to use percentile method (will be covered in Lab 7)

print("95% of resampled medians were between", 
      percentile(2.5, resampled_medians),
      "and",
      percentile(97.5, resampled_medians))


In [ ]:
# Doing it all over again, putting the cells together

sample_from_population = sf.sample(200, with_replacement=False)

resampled_medians = []

for i in np.arange(1000):
    resample = sample_from_population.sample()
    median = np.median(resample.column(2))
    resampled_medians.append(median)

Table().with_column('Resampled median', resampled_medians).hist(0)

# Draw the line and dot
interval_95 = [percentile(2.5, resampled_medians),
               percentile(97.5, resampled_medians)]
plots.plot(interval_95, [0, 0], color='gold', lw=5)
pop_median = np.median(sf.column(2))
plots.scatter(pop_median, 0, color='red', s=400)

## Interval Estimates

In [ ]:
# THE BIG SIMULATION: This one takes several minutes.

# Generate 100 intervals, in the table intervals

def bootstrap_median(sample_from_population, label, repetitions):
    resampled_medians = []
    for i in np.arange(repetitions):
        resample = sample_from_population.sample()
        median = np.median(resample.column(label))
        resampled_medians.append(median)
    return resampled_medians

left_ends = make_array()
right_ends = make_array()

total_comps = sf.select(2)
for i in np.arange(100):
    sample_from_pop = total_comps.sample(200, with_replacement=False)
    medians = bootstrap_median(sample_from_pop, 'Total Compensation', 5000)
    left_ends = np.append(left_ends, percentile(2.5, medians))
    right_ends = np.append(right_ends, percentile(97.5, medians))
    print('Iteration:', i)

intervals = Table().with_columns(
    'Left', left_ends,
    'Right', right_ends
)

In [ ]:
intervals.show(3)


In [ ]:
(intervals
 .where('Left', are.below(pop_median))
 .where('Right', are.above(pop_median))
 .num_rows)

In [ ]:
#visual represenation 
replication_number = np.ndarray.astype(np.arange(1, 101), str)

intervals2 = Table(replication_number).with_rows(make_array(left_ends, right_ends))

plots.figure(figsize=(8,8))
n=100
for i in np.arange(n):
    ends = intervals2.column(i)
    plots.plot(ends, make_array(i+1, i+1), color='gold')
plots.plot(make_array(pop_median, pop_median), make_array(0, n), color='red', lw=2)
plots.xlabel('Median (dollars)')
plots.ylabel('Replication')
plots.title('Population Median and Intervals of Estimates');

back to slides

## Baby Weights

In [ ]:
# http://inferentialthinking.com/notebooks/baby.csv
births = Table.read_table('baby.csv')
births.show(3)

In [ ]:
babies = births.select(0, 1)
babies.show(3)

In [ ]:
ratios = babies.with_column(
    'Ratio BW:GD', babies.column(0) / babies.column(1)
)
ratios.show(3)

In [ ]:
ratios.hist(2)

In [ ]:
np.median(ratios.column(2))

In [ ]:
# 5000 bootstrap samples
# We've taken the median of each of these samples
# Plotted them on the histogram
# We've constructed the 95% confidence interval (in yellow)

resampled_medians = bootstrap_median(ratios, 2, 5000)

interval_95 = make_array(
    percentile(2.5, resampled_medians),
    percentile(97.5, resampled_medians)
)

Table().with_column('Resampled median', resampled_medians).hist(0)
plots.plot(interval_95, [0, 0], color='gold', lw=8)
print('Approximate 95% Bootstrap Confidence Interval for the Population Median')
print(np.round(interval_95, 4))

In [ ]:
interval_80 = make_array(
    percentile(10, resampled_medians),
    percentile(90, resampled_medians)
)

Table().with_column('Resampled median', resampled_medians).hist(0)
plots.plot(interval_80, [0, 0], color='gold', lw=8)
print('Approximate 80% Bootstrap Confidence Interval for the Population Median')

print(np.round(interval_80, 4))

<img src="notbootstrap.PNG" width=600> 

In [ ]:
def bootstrap_mean(sample_from_population, label, repetitions):
    resampled_means = []
    for i in np.arange(repetitions):
        resample = sample_from_population.sample()
        mean = np.mean(resample.column(label))
        resampled_means.append(mean)
    return resampled_means

In [ ]:
def bootstrap_ci_mean(sample_from_population, label, repetitions):
    resampled_means = bootstrap_mean(sample_from_population, label, repetitions)
    
    interval_95 = make_array(
        percentile(2.5, resampled_means),
        percentile(97.5, resampled_means)
    )
    
    Table().with_column('Resampled mean', resampled_means).hist(0)
    plots.plot(interval_95, [0, 0], color='gold', lw=8)
    print('Approximate 95% Bootstrap Confidence Interval for Population Mean:')
    print(np.round(interval_95, 3))

In [ ]:
births.show(3)

In [ ]:
np.median(births.column("Maternal Age"))

In [ ]:
bootstrap_ci_mean(births, 'Maternal Age', 5000)
# how do we interpret that? (iclicker question)

<img src="ci.PNG" width=600> 

In [ ]:
births.hist('Maternal Age')